# RAG 시스템 - 뷰티 코칭 AI (Hybrid: HuggingFace + OpenAI)

이 노트북은 PDF 문서를 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템입니다.

## 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어 지원)
- **LLM**: OpenAI GPT-4o-mini (유료, 고품질, 비전 지원)

## 주요 기능
- PDF 문서 로드 및 전처리
- HuggingFace 다국어 임베딩 및 Chroma DB 저장 (무료!)
- 한글 쿼리로 영어 논문도 검색 가능
- OpenAI GPT-4o-mini 비전 모델을 사용한 이미지+텍스트 질의응답

## 1. 라이브러리 Import

In [27]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import re
import json
import time
import os
import base64
from pathlib import Path
import glob as glob_module

load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print("✅ OpenAI API 키가 설정되었습니다.")

✅ OpenAI API 키가 설정되었습니다.


## 2. 유틸리티 함수 정의

In [28]:
def clean_text(text):
    """텍스트 공백 정리 (PDF OCR 오류 대응 강화)"""
    
    # 1. 단어 내부의 과도한 공백 제거 (가장 공격적)
    # "T r a d i t i o n a l" -> "Traditional"
    # 패턴: 1-2글자 + 공백이 3번 이상 반복
    text = re.sub(r'\b(\w{1,2})\s+(?=\w{1,2}\s+\w)', r'\1', text)
    
    # 2. 남은 단일 문자 사이 공백 제거
    # "t h e" -> "the"
    text = re.sub(r'\b(\w)\s+(\w)\s+(\w)\b', r'\1\2\3', text)
    text = re.sub(r'\b(\w)\s+(\w)\b', r'\1\2', text)
    
    # 3. 특수문자와 단어 사이 불필요한 공백 제거
    # "ﬂ a t" -> "flat"
    text = re.sub(r'(\S)\s+(\S)(?=\s+\S{1,2}\s+|\s+\S{1,2}[,.])', r'\1\2', text)
    
    # 4. 여러 개의 연속된 공백을 하나로
    text = re.sub(r' {2,}', ' ', text)
    
    # 5. 탭을 공백으로
    text = text.replace('\t', ' ')
    
    # 6. 줄바꿈 전후 하이픈 처리 (단어 분리)
    # "dis-\nrupting" -> "disrupting"
    text = re.sub(r'-\s*\n\s*', '', text)
    
    # 7. 3개 이상의 연속된 줄바꿈을 2개로
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # 8. 각 줄의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    
    # 9. 빈 줄과 너무 짧은 줄 제거 (노이즈 제거)
    lines = [line for line in lines if len(line) > 1]
    
    text = '\n'.join(lines)
    
    return text.strip()

def load_single_pdf(pdf_path):
    """단일 PDF 파일 로드 (PyMuPDFLoader 사용)"""
    try:
        loader = PyMuPDFLoader(str(pdf_path))
        docs = loader.load()
        return docs, "PyMuPDFLoader"
    except Exception as e:
        # 에러 정보를 함께 반환
        raise Exception(f"PyMuPDFLoader 실패: {str(e)}")

def load_documents():
    """PDF 문서 로드"""
    
    # data 폴더의 모든 PDF 파일 찾기
    pdf_files = list(Path("data/").rglob("*.pdf"))
    print(f"📁 발견된 PDF 파일: {len(pdf_files)}개\n")
    
    all_documents = []
    failed_files = []
    loader_stats = {}
    
    for pdf_path in pdf_files:
        try:
            print(f"  로딩: {pdf_path.name}...", end=" ")
            docs, loader_name = load_single_pdf(pdf_path)
            all_documents.extend(docs)
            loader_stats[loader_name] = loader_stats.get(loader_name, 0) + 1
            print(f"✅ ({len(docs)}페이지, {loader_name})")
        except Exception as e:
            print(f"❌ 실패: {str(e)}")
            failed_files.append((pdf_path.name, str(e)))
    
    print(f"\n✅ 총 {len(all_documents)}개 페이지 로드 완료")
    print(f"📊 사용된 로더: {loader_stats}")
    
    if failed_files:
        print(f"\n⚠️ {len(failed_files)}개 파일 로드 실패:")
        for filename, error in failed_files:
            print(f"  - {filename}: {error}")
    
    # 텍스트 정리 적용
    print("\n🧹 텍스트 정리 중...")
    for i, doc in enumerate(all_documents):
        original_length = len(doc.page_content)
        doc.page_content = clean_text(doc.page_content)
        cleaned_length = len(doc.page_content)
        
        # 정리 효과 확인 (첫 페이지만)
        if i == 0 and original_length > 0:
            print(f"\n📊 첫 페이지 정리 효과:")
            print(f"  원본 길이: {original_length}자")
            print(f"  정리 후: {cleaned_length}자")
            print(f"  감소율: {(1 - cleaned_length/original_length)*100:.1f}%")
            print(f"\n첫 200자 미리보기:")
            print(doc.page_content[:200])
            print("...")
    
    return all_documents

def chunk_doc(documents, chunk_size=1000, chunk_overlap=200):
    """문서를 청크로 분할 - 개선된 설정
    
    Args:
        chunk_size: 청크 크기 (기본 1000자 - 더 큰 의미 단위 유지)
        chunk_overlap: 청크 간 중복 (기본 200자 - 문맥 연속성 유지)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        # separators 순서 개선: 큰 의미 단위부터 분할
        separators=[
            "\n\n\n",  # 여러 단락 구분
            "\n\n",    # 단락 구분
            "\n",      # 줄바꿈
            "。",      # 한글 마침표
            ". ",      # 영문 문장 끝
            "! ",      # 느낌표
            "? ",      # 물음표
            "；",      # 세미콜론
            "，",      # 쉼표
            " ",       # 공백 (최후의 수단)
            ""         # 문자 단위 (최후의 최후)
        ],
        keep_separator=True,  # 구분자 유지
        length_function=len,
    )
    splits = text_splitter.split_documents(documents)
    
    # 청크 품질 확인
    print(f"\n📊 청크 통계:")
    print(f"  총 청크 수: {len(splits)}")
    if len(splits) > 0:
        avg_length = sum(len(s.page_content) for s in splits) / len(splits)
        min_length = min(len(s.page_content) for s in splits)
        max_length = max(len(s.page_content) for s in splits)
        print(f"  평균 청크 길이: {avg_length:.0f}자")
        print(f"  최소/최대: {min_length}/{max_length}자")
        print(f"\n첫 번째 청크 미리보기 (500자):")
        print(splits[0].page_content[:500])
        print("...")
    
    return splits

def save_documents(documents, output_path):
    """문서를 JSONL 형식으로 저장"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, page in enumerate(documents):
            data = {
                'page': i + 1,
                'text': page.page_content,
                'metadata': {
                    'source': page.metadata.get('source', 'unknown'),
                    'page': page.metadata.get('page', i)
                }
            }
            f.write(json.dumps(data, ensure_ascii=False) + '\n')

def encode_image(image_path):
    """이미지를 base64로 인코딩"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

print("✅ 유틸리티 함수 정의 완료 (개선된 청킹 설정)")

✅ 유틸리티 함수 정의 완료 (개선된 청킹 설정)


## 3. 문서 로드 및 전처리

**이 셀은 한 번만 실행하세요!**

In [29]:
print("문서 로드 중...")
documents = load_documents()
print(f"✅ 총 {len(documents)}개의 문서 로드 완료")

print("\n문서 청킹 중...")
chunked_documents = chunk_doc(documents)
print(f"✅ 총 {len(chunked_documents)}개의 청크 생성 완료")

print("\n문서 저장 중...")
save_documents(chunked_documents, 'tmp.jsonl')
print("✅ tmp.jsonl 저장 완료")

문서 로드 중...
📁 발견된 PDF 파일: 86개

  로딩: 헤어스타일 변화로 인한 감정 치유와 회복 탄력성의 관계에서 자존감의 매개 효과.pdf... ✅ (14페이지, PyMuPDFLoader)
  로딩: 안면 노화 징후 완화를 위한 구강안면 근기능 개입의 효과-PubMed_Central.pdf... ✅ (24페이지, PyMuPDFLoader)
  로딩: 외모·자아수용·퍼펙셔니즘.pdf... ✅ (8페이지, PyMuPDFLoader)
  로딩: 젊음을 위한 얼굴 운동-Scielo.pdf... ✅ (12페이지, PyMuPDFLoader)
  로딩: 남성 헤어커트의 작품제작.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: 성격 인상 정보가 얼굴 매력 판단에 미치는 영향.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: JKSC-2025-31-2-451.pdf... ✅ (12페이지, PyMuPDFLoader)
  로딩: 모던걸 헤어스타일의 구성요소와 사회적 여성성과의 상관성 연구 - 미용학 전공 여대생 관점으로.pdf... ✅ (6페이지, PyMuPDFLoader)
  로딩: 호호바오일의모발보호효과연구.pdf... ✅ (21페이지, PyMuPDFLoader)
  로딩: 비타민 E, 비타민 A와 phytantriol을 함유한 헤어 에센스의 모발의 질에 미치는 영향.pdf... ✅ (10페이지, PyMuPDFLoader)
  로딩: 괴롭힘_사이버폭력–외로움 상호관계 메타리뷰.pdf... ✅ (53페이지, PyMuPDFLoader)
  로딩: 헤어 트리트먼트제로서 단풍잎 추출물 컬러링 모발의 역학적 변화분석.pdf... ✅ (6페이지, PyMuPDFLoader)
  로딩: 헤어커트 시 두상의 분할과 슬라이스 라인에 따른 원랭스 커트와 세임 레이어 커트 혼합의 커트형태변화.pdf... ✅ (12페이지, PyMuPDFLoader)
  로딩: 형태 측정 분석과 딥 러닝 얼굴 매력도 요인 조사에 대한 계산적 접근법(ㅇㄱ)-PubMed_

## 4. 벡터 임베딩 및 저장 (HuggingFace multilingual-e5-large)

**이 셀도 한 번만 실행하세요! 이미 임베딩이 있다면 5번 셀로 바로 가세요.**

**HuggingFace 다국어 모델 사용 - 무료!** ⭐
**한글 쿼리로 영어 논문도 검색 가능!**

In [32]:
print("=" * 50)
print("HuggingFace multilingual-e5-large 임베딩 시작...")
print(f"청크 개수: {len(chunked_documents)}개")
print("⭐ HuggingFace 다국어 모델 사용 - 무료!")
print("⭐ 한글 쿼리로 영어 논문도 검색 가능!")
print("=" * 50)

embedding_start = time.time()


# HuggingFace multilingual-e5-large 임베딩 모델 사용
# 다국어 지원으로 한글 쿼리로 영어 문서 검색 가능
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ 임베딩 모델 로드 완료")
print("📥 벡터 임베딩 생성 중... (시간이 다소 걸릴 수 있습니다)")

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
    persist_directory="./chroma_db_tmp"
)

embedding_end = time.time()
print(f"✅ HuggingFace multilingual-e5-large 임베딩 완료: {embedding_end - embedding_start:.2f}초")
print("=" * 50)

HuggingFace multilingual-e5-large 임베딩 시작...
청크 개수: 4381개
⭐ HuggingFace 다국어 모델 사용 - 무료!
⭐ 한글 쿼리로 영어 논문도 검색 가능!
✅ 임베딩 모델 로드 완료
📥 벡터 임베딩 생성 중... (시간이 다소 걸릴 수 있습니다)
✅ HuggingFace multilingual-e5-large 임베딩 완료: 73.17초


## 5. 기존 벡터스토어 로드 (HuggingFace multilingual-e5-large)

**임베딩을 다시 하지 않고 기존 것을 사용하려면 이 셀을 실행하세요!**

**HuggingFace 다국어 모델 - 무료!** ⭐

In [ ]:
if os.path.exists("./chroma_db_e5"):
    print("기존 HuggingFace multilingual-e5-large 벡터스토어 로드 중...")
    
    embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large",
        model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
        encode_kwargs={'normalize_embeddings': True}
    )
    
    vectorstore = Chroma(
        persist_directory="./chroma_db_tmp",
        embedding_function=embeddings
    )
    print("✅ 벡터스토어 로드 완료")
else:
    print("❌ 벡터스토어가 없습니다. 4번 셀을 먼저 실행하세요.")

기존 HuggingFace multilingual-e5-large 벡터스토어 로드 중...
✅ 벡터스토어 로드 완료


## 6. RAG 체인 설정 (Hybrid: HuggingFace 임베딩 + OpenAI GPT-4o-mini)

**비용 효율적인 하이브리드 접근:**
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어 지원)
- **LLM/Chat**: OpenAI GPT-4o-mini (유료, 고품질, 비전 지원)

In [ ]:
# Retriever 설정
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 중복 제거를 고려해서 많이 가져옴
)

bm25_retriever = BM25Retriever.from_documents(chunked_documents)
bm25_retriever.k = 5

# OpenAI GPT-4o-mini 모델 설정 (비전 지원)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

# --- MultiQuery (LLM 기반 쿼리 확장) ---
multi_query_prompt = PromptTemplate.from_template("""
Generate 3 different search queries that help retrieve relevant academic or scientific information 
related to the following question. 
Write each query on a new line.

Question: {question}
""")

generate_queries = (
    multi_query_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | RunnableLambda(lambda x: x.strip().split("\n"))
)

# --- RRF(Reciprocal Rank Fusion) ---
def rrf_merge(dense_docs, sparse_docs, k=60):
    scores = {}

    def add_results(docs):
        for rank, doc in enumerate(docs):
            doc_id = doc.metadata.get("id", id(doc))
            score = 1 / (k + rank + 1)
            if doc_id not in scores:
                scores[doc_id] = {"doc": doc, "score": 0}
            scores[doc_id]["score"] += score

    add_results(dense_docs)
    add_results(sparse_docs)

    merged = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
    return [item["doc"] for item in merged]


# --- MultiQuery + Hybrid Retrieval ---
def hybrid_multi_retrieve(queries):
    dense_all = []
    sparse_all = []

    for q in queries:
        dense_all.extend(retriever.invoke(q))
        sparse_all.extend(bm25_retriever.invoke(q))

    merged = rrf_merge(dense_all, sparse_all)
    return merged


hybrid_multi_retriever = (
    generate_queries
    | RunnableLambda(hybrid_multi_retrieve)
)


# LCEL 방식으로 체인 구성
def deduplicate_docs(docs):
    """중복 문서 제거"""
    seen = set()
    unique_docs = []
    for doc in docs:
        # 문서 내용으로 중복 체크
        content_hash = hash(doc.page_content)
        if content_hash not in seen:
            seen.add(content_hash)
            unique_docs.append(doc)
    return unique_docs[:7]  # 최종적으로 7개만 반환

def format_docs(docs):
    """중복 제거 후 포맷팅"""
    unique_docs = deduplicate_docs(docs)
    return "\n\n".join(doc.page_content for doc in unique_docs)

def create_message_with_image(inputs):
    """이미지와 텍스트를 함께 메시지로 생성"""
    context = inputs["context"]
    question = inputs["question"]
    image_data = inputs.get("image")

    # 프롬프트 텍스트 구성 (한글 답변 강제)
    prompt_text = f"""당신은 다음 3명의 전문가가 통합된 고급 이미지·뷰티 코칭 AI입니다.
- 1. 헤어 스타일리스트: 헤어 형태, 볼륨, 질감, 스타일 방향성, 머릿결 관리 등 분석
- 2. 피부 관리 전문가: 피부 톤, 수분·유분 상태, 결, 잡티, 광택 등 관찰
- 3. 얼굴 윤곽 디자이너: 눈·코·입 비율, 턱선, 볼살, 이마 곡선, 대칭, 입체감 등 분석

[분석 방식 지침]
1. 사용자가 제공한 성별, 직업, 상황 설명을 기반으로 한다.
2. 관찰은 반드시 시각적·감각적 요소 중 최소 3가지 이상을 활용하고 디테일을 풍부하게 묘사하여, 독자가 장면을 그릴 수 있게 작성한다.
3. 부정적인 표현을 피하고 개선 가능성을 중심으로 서술한다.
4. 개선 팁은 오늘 실천 가능한 구체적 행동이나 셀프 케어 방법만 포함해야 하며, 추상적 조언은 금지된다.
5. 따뜻하고 격려적인 어조를 유지하며, 객관적이되 전문적으로 작성한다.
6. 전문 용어를 적절히 사용하되 이해하기 쉬운 표현으로 설명한다.

[중요: 질문 답변 우선 원칙]
- 사용자가 특정 질문(예: "염색 추천해줘", "헤어스타일 추천해줘")을 한 경우, 반드시 해당 질문에 대한 구체적인 답변을 먼저 제공해야 합니다.
- 질문과 관련된 카테고리(Hair/Skin/Contour)의 status에 질문에 대한 답변을 포함시키세요.
- 예시: "염색 추천해줘" → Hair.status에 "현재 [기존 헤어 상태]. 화사한 느낌을 위해 [구체적 염색 색상과 톤] 추천드립니다" 형식으로 작성

[논문 인용 원칙] ⭐ 중요 ⭐
- 참고 문서에 관련 논문 내용이 있으면 **반드시 직접 인용**하세요.
- 인용 시 따옴표("")를 사용하고, 구체적인 연구 결과, 수치, 전문 용어를 포함하세요.
- 예시: "연구에 따르면 '열펌 시술 시 180도 이상의 고온은 모발의 단백질 변성을 일으킬 수 있다'고 합니다"
- 예시: "논문에서는 '시스틴 결합이 부분적으로 절단되고 각각 수소원자와 결합되어 고분자의 시스테인을 만들게 된다'고 설명합니다"
- status와 improvement_tips 모두에서 논문 내용을 적극적으로 인용하여 전문성을 높이세요.

[분석 카테고리]
1. 헤어(Hair)
2. 피부(Skin)
3. 윤곽(Contour)

[참고 문서]
{context}

[사용자 질문]
{question}

[Output JSON Example]
{{{{
  "Appearance_Coaching": {{{{
    "Hair": {{{{
      "status": "헤어의 현재 상태에 대한 간결한 요약 설명. 만약 사용자가 헤어 관련 질문(염색, 스타일 등)을 했다면, 여기에 구체적인 추천을 포함하여 작성. 참고 문서의 논문 내용이 있다면 따옴표를 사용하여 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1 (논문 인용 포함 가능)",
        "오늘 실천 가능한 구체적인 행동 2 (논문 인용 포함 가능)"
      ]
    }}}},
    "Skin": {{{{
      "status": "피부의 현재 상태에 대한 간결한 요약 설명. 만약 사용자가 피부/메이크업 관련 질문을 했다면, 여기에 구체적인 추천을 포함하여 작성. 논문 내용 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1 (논문 인용 포함 가능)",
        "오늘 실천 가능한 구체적인 행동 2 (논문 인용 포함 가능)"
      ]
    }}}},
    "Contour": {{{{
      "status": "윤곽의 현재 상태에 대한 간결한 요약 설명. 논문 내용 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1 (논문 인용 포함 가능)",
        "오늘 실천 가능한 구체적인 행동 2 (논문 인용 포함 가능)"
      ]
    }}}}
  }}}}
}}}}

**중요 규칙:**
1. 답변은 반드시 한글로만 작성해야 합니다. 영어는 절대 사용하지 마세요.
2. 출력은 반드시 위 JSON 형식으로 하며, 개선 팁은 반드시 오늘 실천 가능한 구체적 행동으로 작성하세요.
3. 사용자 질문이 특정 주제(염색, 헤어스타일, 메이크업 등)에 대한 것이면, 해당 카테고리의 status에 질문에 대한 구체적인 답변을 우선적으로 포함하세요.
4. status는 시각적·감각적 요소를 3가지 이상 포함하되, 질문에 대한 답변을 먼저 작성하고 현재 상태 분석을 추가하세요.
5. **참고 문서에 논문 내용이 있으면 반드시 따옴표("")를 사용하여 직접 인용하고, 연구 결과나 전문 지식을 적극 활용하세요.**

답변:"""

    # 이미지가 있는 경우 멀티모달 메시지 생성
    if image_data:
        return [HumanMessage(
            content=[
                {"type": "text", "text": prompt_text},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
                }
            ]
        )]
    else:
        # 이미지가 없는 경우 텍스트만 전달
        return [HumanMessage(content=prompt_text)]

qa_chain = (
    {
        "context": lambda x: format_docs(hybrid_multi_retriever.invoke({"question": x["question"]})),
        "question": lambda x: x["question"],
        "image": lambda x: x.get("image")
    }
    | RunnableLambda(create_message_with_image)
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인 설정 완료 (이미지 지원, 중복 제거 기능 추가)")

✅ RAG 체인 설정 완료 (이미지 지원, 중복 제거 기능 추가)


In [39]:
test_query = "I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem."
hybrid_multi_retriever.invoke(test_query)

AttributeError: 'AIMessage' object has no attribute 'strip'

[Document(metadata={'trapped': '', 'producer': 'Adobe PDF Library 17.0', 'keywords': '', 'creationDate': "D:20251009095737+03'00'", 'title': '', 'source': 'data/AA–불안·우울 연관 메타분석.pdf', 'modDate': "D:20251009095737+03'00'", 'creationdate': '2025-10-09T09:57:37+03:00', 'file_path': 'data/AA–불안·우울 연관 메타분석.pdf', 'total_pages': 5, 'page': 4, 'format': 'PDF 1.4', 'creator': 'Adobe InDesign 19.5 (Macintosh)', 'moddate': '2025-10-09T09:57:37+03:00', 'author': '', 'subject': ''}, page_content='1040-50.\n7.\nLauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.\nPrevalence and odds of depressive and anxiety disorders and symptoms in\nchildren and adults with alopecia areata: a systematic review and metaanalysis. JAMA Dermatol. 2023; 159(3): 281-8.\n8.\nKurd SK, Troxel AB, Crits-Christoph P, Gelfand JM. The risk of depression,\nanxiety, and suicidality in patients with psoriasis: a population-based cohort\nstudy. Arch Dermatol. 2010; 146(8): 891-5.\n9.\nAlmutawa YM, AlGhareeb M, Bhattarai E, Aljalahma J. Investigation of the\nimpact of atopic dermatitis (AD) on stress, depression, anxiety, and suicidal\nideation: a systematic review and meta-analysis. Cureus. 2024; 16(6):\ne63376.\n10.  Nerurkar L, Siebert S, McInnes IB, Cavanagh J. Rheumatoid arthritis and\ndepression: an inflammatory perspective. Lancet Psychiatry. 2019; 6(2): 16473.\n11.  Martin-Subero M, Anderson G, Kanchanatawan B, Berk M, Maes M.'),
 Document(metadata={'creationDate': "D:20251009095737+03'00'", 'keywords': '', 'total_pages': 5, 'source': 'data/AA–불안·우울 연관 메타분석.pdf', 'title': '', 'subject': '', 'format': 'PDF 1.4', 'page': 4, 'creationdate': '2025-10-09T09:57:37+03:00', 'author': '', 'file_path': 'data/AA–불안·우울 연관 메타분석.pdf', 'moddate': '2025-10-09T09:57:37+03:00', 'modDate': "D:20251009095737+03'00'", 'trapped': '', 'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.5 (Macintosh)'}, page_content='17.  Kutlu Ö, Aktaş H, İmren IG, Metin A. Short-term stress-related increasing\ncases of alopecia areata during the COVID-19 pandemic. J Dermatolog Treat.\n2022; 33(2): 1177.\n18.  Ferentinos P, Kalogeropoulou E, Pappa G, Antoniou A, Bozi E, Kyprianou\nM, et al. Assessing the role of stressful life events in the induction and\nrecurrence of alopecia areata: a case-control study. JAm Acad Dermatol.\n2022; 87(5): 1215-7.\n19.  Taheri R, Behnam B, Tousi JA, Azizzade M, Sheikhvatan MR. Triggering role\nof stressful life events in patients with alopecia areata. Acta Dermatovenerol\nCroat. 2012; 20(4): 246-50.\n20.  Russiello F, Arciero G, Decaminada F, Corona R, Ferrigno L, Fucci M, et\nal. Stress, attachment and skin disease: a case-control study. J Eur Acad\nDermatol Venereol. 2006; 5(3): 234-9\n21.  Vélez-Muñiz RDC, Peralta-Pedrero ML, Jurado-Santa Cruz F, Morales-Sánchez\nMA. Psychological profile and quality of life of patients with alopecia areata.\nSkin Appendage Disord. 2019; 5(5): 293-98.'),
 Document(metadata={'source': 'data/원형탈모(AA) 환자 삶의 질·정서 영향.pdf', 'creationdate': '2025-07-19T12:37:48+02:00', 'file_path': 'data/원형탈모(AA) 환자 삶의 질·정서 영향.pdf', 'creator': 'Adobe InDesign 20.2 (Macintosh)', 'page': 6, 'modDate': "D:20250719123748+02'00'", 'trapped': '', 'subject': '', 'moddate': '2025-07-19T12:37:48+02:00', 'format': 'PDF 1.4', 'author': '', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20250719123748+02'00'", 'title': '', 'keywords': '', 'total_pages': 7}, page_content='and management. J Dtsch Dermatol Ges 2022; 20: 59–90.\nhttps://doi.org/10.1111/ddg.14689\n2. Gilhar A, Etzioni A, Paus R. Alopecia areata. N Engl J\nMed 2012; 366: 1515–1525. https://doi.org/10.1056/\nNEJMra1103442\n3. Aldhouse NVJ, Kitchen H, Knight S, Macey J, Nunes FP, Dutronc Y, et al. “‘You lose your hair, what’s the big deal?’ I was so\nembarrassed, I was so self-conscious, I was so depressed:”\na qualitative interview study to understand the psychosocial\nburden of alopecia areata. J Patient Rep Outcomes 2020; 4:\n76. https://doi.org/10.1186/s41687-020-00240-74. Rencz F, Gulacsi L, Pentek M, Wikonkal N, Baji P, Brodszky\nV. Alopecia areata and health-related quality of life: a systematic review and meta-analysis. BrJ Dermatol 2016; 175:\n561–571. https://doi.org/10.1111/bjd.14497\n5. King BA, Senna MM, Ohyama M, Tosti A, Sinclair RD, Ball\nS, et al. Defining severity in alopecia areata: current perspectives and a multidimensional framework. Dermatol Ther'),
 Document(metadata={'modDate': "D:20251009095737+03'00'", 'author': '', 'moddate': '2025-10-09T09:57:37+03:00', 'creationdate': '2025-10-09T09:57:37+03:00', 'subject': '', 'format': 'PDF 1.4', 'title': '', 'total_pages': 5, 'source': 'data/AA–불안·우울 연관 메타분석.pdf', 'page': 4, 'creator': 'Adobe InDesign 19.5 (Macintosh)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20251009095737+03'00'", 'file_path': 'data/AA–불안·우울 연관 메타분석.pdf', 'trapped': '', 'keywords': ''}, page_content='MA. Psychological profile and quality of life of patients with alopecia areata.\nSkin Appendage Disord. 2019; 5(5): 293-98.\n22.  Bewley A, Figueras-Nart I, Zhang J, Guerreiro M, Tietz N, Chtourou S, et al.\nPatient-reported burden of severe alopecia areata: first results from the\nmultinational alopecia areata unmet need survey. Clin Cosmet Investig\nDermatol. 2024; 17: 751-61.\n23.  Clemmesen MER, Gren ST, Frøstrup AG, Thomsen SF, Egeberg A, Thein D.\nPsychosocial and mental impact of alopecia areata: analysis of the Danish\nSkin Cohort. J Eur Acad Dermatol Venereol. 2025; 39(3): 688-97.'),
 Document(metadata={'creationdate': '2025-10-09T09:57:37+03:00', 'page': 0, 'format': 'PDF 1.4', 'modDate': "D:20251009095737+03'00'", 'file_path': 'data/AA–불안·우울 연관 메타분석.pdf', 'producer': 'Adobe PDF Library 17.0', 'total_pages': 5, 'keywords': '', 'creationDate': "D:20251009095737+03'00'", 'author': '', 'trapped': '', 'title': '', 'source': 'data/AA–불안·우울 연관 메타분석.pdf', 'moddate': '2025-10-09T09:57:37+03:00', 'creator': 'Adobe InDesign 19.5 (Macintosh)', 'subject': ''}, page_content='Pınar Özdemir Çetinkaya1, Ömer Kutlu2\nEvaluation of Quality of Life, Anxiety and Depression in\nPatients with Alopecia Areata: A Prospective Case-Control\nStudy\nDOI: 10.4274/cjms.2025.2025-1\nINTRODUCTION\nAlopecia areata (AA) isan immune response-driven, scarless hair disease\nspecified by destruction of hair follicles byan inflammatory response\nresulting from the loss of immune privilege.1It may manifest as one\nor more alopecic patches on the scalp, as well as alopecia totalis or\nalopecia universalis. It can also involve the facial region, leading to hair\nloss in the eyebrows, eyelashes, and beard.2,3 The progression ofAAis\nunpredictable, asit may either resolve spontaneously or persist for a long\ntime, even a lifetime. Given that the duration of the current episode isa\nkey prognostic factor, AA was classified as acute (≤12 months) or chronic\n(>12 months).4 The condition poses a cosmetic concern and impacts')]

## 7. 질의응답 실행 (텍스트만)

**이미지 없이 텍스트만 사용하는 경우**

In [ ]:
# 질문을 여기서 수정하세요
query = "I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem."

print("=" * 50)
print(f"질문: {query}")
print("=" * 50)
print("\nRAG 쿼리 시작... (OpenAI API 호출 중)")
invoke_start = time.time()

result = qa_chain.invoke({"question": query})

invoke_end = time.time()
print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
print("=" * 50)

print("\n📝 답변:")
print(result)

질문: 28일동안 샴푸를 사용하여 머리를 감은상태야 내 머리 상태를 평가해줘

RAG 쿼리 시작... (OpenAI API 호출 중)
✅ RAG 쿼리 완료: 6.14초

📝 답변:
{
  "Appearance_Coaching": {
    "Hair": {
      "status": "현재 머리카락은 28일 동안 샴푸로 관리된 상태로, 약간의 기름기와 함께 건강한 볼륨을 유지하고 있습니다. 머리결은 부드러움이 느껴지며, 전체적으로 자연스러운 윤기가 돌고 있습니다. 하지만 두피에 약간의 건조함이 느껴질 수 있습니다. 이러한 상태를 유지하면서도 더 나은 관리가 가능합니다.",
      "improvement_tips": [
        "온수로 헹군 후 찬물로 마무리하여 머리카락의 큐티클을 닫아 윤기를 더하세요.",
        "두피에 수분을 공급하기 위해 수분 보충 효과가 있는 두피 전용 에센스를 사용해 보세요."
      ]
    },
    "Skin": {
      "status": "피부는 전반적으로 건강한 상태를 유지하고 있으나, 가볍게 유분이 올라올 수 있습니다. 피부 톤은 고르게 보이며, 약간의 수분 부족이 느껴질 수 있습니다. 잡티는 없는 편이며, 자연스러운 광택이 있습니다.",
      "improvement_tips": [
        "하루에 두 번 수분 크림을 충분히 발라 피부의 수분을 보충하세요.",
        "자외선 차단제를 꼭 사용하여 피부를 보호하세요."
      ]
    },
    "Contour": {
      "status": "얼굴의 윤곽은 전체적으로 균형 잡힌 상태입니다. 턱선이 부드럽고 볼살은 적당하여 입체감이 느껴집니다. 이마와 턱의 곡선이 자연스럽고, 얼굴의 대칭도 잘 유지되고 있습니다.",
      "improvement_tips": [
        "하루에 5분 정도 얼굴 마사지로 혈액 순환을 촉진해 보세요.",
        "자주 미소 지으며 얼굴 근육을 풀어주어 자연스러운 윤곽

## 8. 질의응답 실행 (이미지 포함)

**이미지와 함께 질문하는 경우**

**주의: GPT-4o 비전 모델은 비용이 더 많이 발생합니다!**

In [ ]:
# 이미지 경로와 질문을 수정하세요
image_path = "sample.png"
query = "I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem."

if Path(image_path).exists():
    print("=" * 50)
    print(f"이미지: {image_path}")
    print(f"질문: {query}")
    print("=" * 50)
    print("\n이미지 인코딩 중...")
    
    image_base64 = encode_image(image_path)
    print("✅ 이미지 인코딩 완료")
    
    print("\nRAG 쿼리 시작... (OpenAI GPT-4o Vision API 호출 중)")
    invoke_start = time.time()
    
    result = qa_chain.invoke({
        "question": query,
        "image": image_base64
    })
    
    invoke_end = time.time()
    print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
    print("=" * 50)
    
    print("\n📝 답변:")
    print(result)
else:
    print(f"❌ 이미지 파일을 찾을 수 없습니다: {image_path}")

이미지: sample.png
질문: I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem.

이미지 인코딩 중...
✅ 이미지 인코딩 완료

RAG 쿼리 시작... (OpenAI GPT-4o Vision API 호출 중)
✅ RAG 쿼리 완료: 71.82초

📝 답변:
```json
{
  "Appearance_Coaching": {
    "Hair": {
      "status": "현재 머리카락은 일부 탈모가 있는 상태입니다. 모발이 고르지 않게 자라나며, 두피가 보이는 부분이 있습니다. 이러한 상태는 자신감을 떨어뜨릴 수 있지만, 적절한 관리로 상태를 개선할 수 있습니다.",
      "improvement_tips": [
        "탈모 관리에 도움을 줄 수 있는 비타민 B 복합체와 아미노산이 포함된 영양제를 섭취해보세요. 연구에 따르면 '비타민 B는 모발 성장에 긍정적인 영향을 미칠 수 있다'고 합니다.",
        "두피 마사지를 통해 혈액 순환을 촉진하고 모발 성장을 도울 수 있습니다. 매일 5-10분 동안 두피를 부드럽게 마사지해보세요."
      ]
    },
    "Skin": {
      "status": "두피와 얼굴 피부는 탈모와 관련하여 민감할 수 있습니다. 자극 없이 깨끗하게 관리하는 것이 중요합니다. 피부의 수분과 유분 상태를 점검하여 적절한 제품을 선택하세요.",
      "improvement_tips": [
        "자극이 적은 샴푸와 컨디셔너를 사용하여 두피의 건강을 유지하고, 매주 1-2회 두피 스크럽을 해보세요. 이는 각질 제거에 도움을 줄 수 있습니다.",
        "하이드레이팅 마스크를 사용하여 얼굴 피부의 수분을 보충하세요. 이는 피부의 탄력을 높이고, 건강한 광택을 줄 수 있습니다."
      ]
    

In [ ]:
# 마지막 질문에 대한 참고 문서 확인
retrieved_docs = retriever.invoke(query)

print("📚 참고 문서:")
print("=" * 50)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[문서 {i}]")
    print(doc.page_content)
    print("-" * 50)